In [1]:
import findspark
findspark.init()
findspark.find()

'/home/nishi/.local/lib/python3.10/site-packages/pyspark'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = ( 
    SparkSession.builder.appName("TaxiOperationDataFrameApp")
    .master("local[4]")
    .config("spark.dynamicAllocation.enabled", "false")
    .config("spark.sql.adaptive.enabled", "false")
    .getOrCreate()
    )
sc = spark.sparkContext
spark

23/08/13 21:07:57 WARN Utils: Your hostname, nishi-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.1.40 instead (on interface enp0s31f6)
23/08/13 21:07:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/13 21:07:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from IPython.display import *
display(HTML("<style>pre {white-space: pre !important; }</style>"))

In [11]:
# Create RDD with Complex Type
data = [
    [1, "Neha", 10000],
    [2, "Steve", 20000],
    [3, "Kari", 30000],
    [4, "Ivan", 40000],
    [5, "Mohit", 50000],
]
employeeRDD = sc.parallelize(data)

In [12]:
# Create DataFrame from RDD
employeesDF = employeeRDD.toDF()
employeesDF.show()

+---+-----+-----+
| _1|   _2|   _3|
+---+-----+-----+
|  1| Neha|10000|
|  2|Steve|20000|
|  3| Kari|30000|
|  4| Ivan|40000|
|  5|Mohit|50000|
+---+-----+-----+



In [13]:
# Define Column name to data frame
employeesDF = employeesDF.toDF("Id", "Name", "Salary")
employeesDF.show()

+---+-----+------+
| Id| Name|Salary|
+---+-----+------+
|  1| Neha| 10000|
|  2|Steve| 20000|
|  3| Kari| 30000|
|  4| Ivan| 40000|
|  5|Mohit| 50000|
+---+-----+------+



In [14]:
# Print Data frame Schema
employeesDF.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Name: string (nullable = true)
 |-- Salary: long (nullable = true)



In [17]:
#Read Yellowtaxis csv file to create DataFrame
yellowTaxiDF = (spark.read.option("header", "true").csv("/home/nishi/PycharmProjects/pythonProject/pySpark/DataFiles/YellowTaxis_202210.csv"))

In [18]:
# Show the Data
yellowTaxiDF.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1|2022-10-01T05:33:...| 2022-10-01T05:48:...|            1.0|          1.7|       1.0|                 N|         249|         107|           1|        9.5|  3.0|    0.5|      2.6

In [20]:
# Print Schema
yellowTaxiDF.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)
 |-- airport_fee: string (nullable = true)



In [21]:
# Use tab delimeter to read GreenTaxi file
greenTaxiDF = (spark
              .read
              .option("header", "true")
              .option("delimiter", "\t")
              .csv("/home/nishi/PycharmProjects/pythonProject/pySpark/DataFiles/GreenTaxis_*.csv"))
greenTaxiDF.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorId|lpep_pickup_datetime|lpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2|2022-10-01T06:08:...| 2022-10-01T06:21:...|            1.0|         2.47|       1.0|                 N|         256|         225|         1.0|       11.5|  0.5|    0.5|      2.5

In [24]:
# Read JSON files
paymentTypesDF = (
                    spark
                    .read
                    .json("/home/nishi/PycharmProjects/pythonProject/pySpark/DataFiles/PaymentTypes.json")
)
paymentTypesDF.show()

+-----------+-------------+
|PaymentType|PaymentTypeID|
+-----------+-------------+
|Credit Card|            1|
|       Cash|            2|
|  No Charge|            3|
|    Dispute|            4|
|    Unknown|            5|
|Voided Trip|            6|
+-----------+-------------+



In [25]:
#Read Yellowtaxis csv file to create DataFrame
yellowTaxiDF = (spark.read.option("header", "true")
                .option("inferSchema", "true")
                .csv("/home/nishi/PycharmProjects/pythonProject/pySpark/DataFiles/YellowTaxis_202210.csv"))
yellowTaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [28]:
# Create schema to yellow Taxi
yellowTaxiSchema = (
StructType([
StructField("VendorID" , IntegerType() , True),
StructField("tpep_pickup_datetime" , TimestampType() , True),
StructField("tpep_dropoff_datetime" , TimestampType() , True),
StructField("passenger_count" , DoubleType() , True),
StructField("trip_distance" , DoubleType() , True),
StructField("RatecodeID" , DoubleType() , True),
StructField("store_and_fwd_flag" , StringType() , True),
StructField("PULocationID" , IntegerType() , True),
StructField("DOLocationID" , IntegerType() , True),
StructField("payment_type" , IntegerType() , True),
StructField("fare_amount" , DoubleType() , True),
StructField("extra" , DoubleType() , True),
StructField("mta_tax" , DoubleType() , True),
StructField("tip_amount" , DoubleType() , True),
StructField("tolls_amount" , DoubleType() , True),
StructField("improvement_surcharge" , DoubleType() , True),
StructField("total_amount" , DoubleType() , True),
StructField("congestion_surcharge" , DoubleType() , True),
StructField("airport_fee" , DoubleType() , True)
])
)

In [30]:
#Create DataFrame using existing dataframe
yellowtaxiDF = (
    spark.read.option("header", "true")
               .schema(yellowTaxiSchema)
               .csv("/home/nishi/PycharmProjects/pythonProject/pySpark/DataFiles/YellowTaxis_202210.csv"))
yellowtaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [36]:
#Read Multiline JSON file
taxiBasesDF = (spark
               .read
               .option("multiline", "true")
               .json("/home/nishi/PycharmProjects/pythonProject/pySpark/DataFiles/TaxiBases.json"))
taxiBasesDF.show(truncate=False)

+-----------------------------------------------------------+----------+--------------------------------------+------------------------------------------------+--------------+------------+----------------+--------+---------------------------+
|Address                                                    |Date      |Entity Name                           |GeoLocation                                     |License Number|SHL Endorsed|Telephone Number|Time    |Type of Base               |
+-----------------------------------------------------------+----------+--------------------------------------+------------------------------------------------+--------------+------------+----------------+--------+---------------------------+
|{636, NEW YORK, 10001, NY, WEST   28 STREET}               |08/15/2019|VIER-NY,LLC                           |{40.75273, (40.75273, -74.006408), -74.006408}  |B02865        |No          |6466657536      |18:03:31|BLACK CAR BASE             |
|{131, BRONX, 10468, NY, KIN

In [37]:
taxiBasesDF.printSchema()

root
 |-- Address: struct (nullable = true)
 |    |-- Building: string (nullable = true)
 |    |-- City: string (nullable = true)
 |    |-- Postcode: long (nullable = true)
 |    |-- State: string (nullable = true)
 |    |-- Street: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Entity Name: string (nullable = true)
 |-- GeoLocation: struct (nullable = true)
 |    |-- Latitude: double (nullable = true)
 |    |-- Location: string (nullable = true)
 |    |-- Longitude: double (nullable = true)
 |-- License Number: string (nullable = true)
 |-- SHL Endorsed: string (nullable = true)
 |-- Telephone Number: long (nullable = true)
 |-- Time: string (nullable = true)
 |-- Type of Base: string (nullable = true)



In [38]:
yellowTaxiAnalyzedDF = (yellowTaxiDF.describe("passenger_count", "trip_distance"))
yellowTaxiAnalyzedDF.show()

+-------+------------------+-----------------+
|summary|   passenger_count|    trip_distance|
+-------+------------------+-----------------+
|  count|           3542392|          3675412|
|   mean|1.3846934500755421|6.206976298167358|
| stddev|0.9302303297407405|640.8236808320215|
|    min|               0.0|              0.0|
|    max|               9.0|        389678.46|
+-------+------------------+-----------------+



In [43]:
# Display the count before operation
print("Before operation = "+ str(yellowTaxiDF.count()))

yellowTaxiDF = (yellowTaxiDF.where("passenger_count > 0")
.filter(col("trip_distance") > 0.0))

# Display the count after operation

print("After operation = "+ str(yellowTaxiDF.count()))

Before operation = 3472140


After operation = 3422296


In [44]:
# Display the count before operation
print( "Before operation = "+ str(yellowTaxiDF.count()))

yellowTaxiDF = (
                yellowTaxiDF.dropDuplicates()
                )
# Display the count after operation
print( "After operation = "+ str(yellowTaxiDF.count()))

Before operation = 3422296


After operation = 3422295


In [48]:
# Display the count before operation
print( "Before operation = "+ str(yellowTaxiDF.count()))

yellowTaxiDF = (
                yellowTaxiDF.where("tpep_pickup_datetime >= '2022-10-01' AND tpep_dropoff_datetime < '2022-11-01'")
                )
# Display the count after operation
print( "After operation = "+ str(yellowTaxiDF.count()))

Before operation = 3422295


After operation = 3393897


In [51]:
# Cleanup data using DataQuality checks

yellowtaxiDF = (yellowTaxiDF
               .where ("passenger_count > 0")
               .filter(col("trip_distance") > 0.0)
               .na.drop('all')
               #.na.fill(defaultValueMap)
               .dropDuplicates()
               .where("tpep_pickup_datetime >= '2022-10-01' AND tpep_dropoff_datetime < '2022-11-01'"))

yellowtaxiDF.count()

3393897

In [56]:
# change DataFrame
yellowTaxiDF = (yellowTaxiDF
               # Select only limites columns
               .select(
               "VendorID",
               col("passenger_count").cast(IntegerType()),
               column("trip_distance").alias("TripDistance"),
               yellowTaxiDF.tpep_pickup_datetime,
               "tpep_dropoff_datetime",
               "PULocationID",
               "DOLocationID",
               "RatecodeID",
               "total_amount",
               "payment_type"))
yellowTaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)



In [57]:
# change DataFrame
yellowTaxiDF = (yellowTaxiDF
               .withColumnRenamed("passenger_count", "PassengerCount")
               .withColumnRenamed("tpep_pickup_datetime", "PickupTime")
               .withColumnRenamed("tpep_dropoff_datetime", "DropTime")
               .withColumnRenamed("PULocationID", "PickupLocationId")
               .withColumnRenamed("DOLocationID", "DropLocationId")
               .withColumnRenamed("total_amount", "TotalAmount")
               .withColumnRenamed("payment_type", "PaymentType"))
yellowTaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = true)



In [59]:
#Create derived column for year , month and day
yellowTaxiDF = (yellowTaxiDF
               .withColumn("TripYear", year(col("PickupTime")))
               .select("*",
                      expr("month(PickupTime) AS TripMonth"),
                      dayofmonth(col("PickupTime")).alias("Tripday")
                      ))

yellowTaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = true)
 |-- TripYear: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- Tripday: integer (nullable = true)



In [60]:
#Option 1
yellowTaxiDF = (yellowTaxiDF
               .withColumn("TripTimeinMinutes",
                           round((unix_timestamp(col("DropTime"))
                                 -unix_timestamp(col("PickupTime")))/60)
                          ))

yellowTaxiDF.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PassengerCount: integer (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- PaymentType: integer (nullable = true)
 |-- TripYear: integer (nullable = true)
 |-- TripMonth: integer (nullable = true)
 |-- Tripday: integer (nullable = true)
 |-- TripTimeinMinutes: double (nullable = true)

